<a href="https://colab.research.google.com/github/le11nguyen/RAG-LLMS-Demo/blob/main/RAG_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers==4.41.2
!pip install -q bitsandbytes==0.43.1
!pip install -q accelerate==0.31.0
!pip install -q langchain==0.2.5
!pip install -q langchainhub==0.1.20
!pip install -q langchain-chroma==0.1.1
!pip install -q langchain-community==0.2.5
!pip install -q langchain_huggingface==0.0.3
!pip install -q python-dotenv==1.0.1
!pip install -q pypdf==4.2.0
!pip install -q numpy==1.24.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [7]:
import torch

from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface.llms import HuggingFacePipeline

from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.chains import ConversationalRetrievalChain

from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub

In [3]:
Loader = PyPDFLoader
FILE_PATH = "/content/cam_nang_dua_don_hoc_sinh.pdf"
loader = Loader(FILE_PATH)
documents = loader.load()

print("Number of documents: ", len(documents))
documents[0]

Number of documents:  46


Document(metadata={'source': '/content/cam_nang_dua_don_hoc_sinh.pdf', 'page': 0}, page_content='1 \n  \n \n \n \n \n \n  \n \n \n   \n \n \nTRƯ ỜNG PH Ổ THÔNG LIÊN C ẤP \nOLYMPIA  \n \n \nCẨM NANG  \nĐƯA ĐÓN H ỌC SINH  \n \n \n \n \n \n \n \n \n \n \n Tận tâm – Ân c ần – Nhiệt tình – Chu đáo  \n')

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                               chunk_overlap=100)

In [5]:
docs = text_splitter.split_documents(documents)

print("Number of mini-documents: ", len(docs))
docs[0]

Number of mini-documents:  75


Document(metadata={'source': '/content/cam_nang_dua_don_hoc_sinh.pdf', 'page': 0}, page_content='1 \n  \n \n \n \n \n \n  \n \n \n   \n \n \nTRƯ ỜNG PH Ổ THÔNG LIÊN C ẤP \nOLYMPIA  \n \n \nCẨM NANG  \nĐƯA ĐÓN H ỌC SINH  \n \n \n \n \n \n \n \n \n \n \n Tận tâm – Ân c ần – Nhiệt tình – Chu đáo')

In [8]:
embedding = HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
vector_db = Chroma.from_documents(documents=docs,
                                  embedding=embedding)

retriever = vector_db.as_retriever()

QUERY = "Các nội dung chính cần biết trong việc đưa đón học sinh"
result = retriever.invoke(QUERY)

print("Number of relevant documents: ", len(result))
result[0]

Number of relevant documents:  4


Document(metadata={'page': 9, 'source': '/content/cam_nang_dua_don_hoc_sinh.pdf'}, page_content='10 \n - Biết quan tâm đ ến những người xung quanh.  \n- Luôn cởi mở tiếp nhận ý kiến đóng góp, liên t ục hoàn thi ện bản thân và có tinh th ần \nhọc hỏi để chất lượng dịch vụ ngày một tốt hơn.  \n3. Giọng đi ệu và l ời nói  \n- Giọng nói có âm lư ợng vừa phải, đủ nghe. Nói có ng ắt, nghỉ, có điểm dừng, điểm nhấn \nrõ ràng, tròn vành, r õ chữ, không nói ng ọng.  \n- Nói đầy đủ cả câu (câu có ch ủ ngữ, vị ngữ), có đủ các đại từ nhân xưng (không nói \ntrống không). Thư ờng xuyên s ử dụng những câu có ý nghĩa tích c ực trong giao ti ếp. \nTuyệt đối không nói t ục, chửi thề. Hạn chế dùng từ địa phương.  \nIII. CÁC QUY  ĐỊNH LAO Đ ỘNG \n1. Thời gian làm vi ệc \nKhung  giờ làm việc của GSV/LX:  \n• Buổi sáng trong kho ảng từ 6h00 đến 8h00  \n• Buổi chiều trong kho ảng từ 15h30 đến 18h00   \n2. Quy  định v ề kỷ luật lao đ ộng \n2.1.  Chấm công   \n- Sau khi th ực hiện công vi ệc cần ký xác nh ận và

In [10]:
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

MODEL_NAME = "lmsys/vicuna-7b-v1.5"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=nf4_config,
    low_cpu_mem_usage=True
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    pad_token_id=tokenizer.eos_token_id,
    device_map="auto"
)

llm = HuggingFacePipeline(
    pipeline=model_pipeline,
)

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/162 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

In [11]:
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

USER_QUESTION = "Các nội dung chính cần biết trong việc đưa đón học sinh là gì?"
output = rag_chain.invoke(USER_QUESTION)
output

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


"Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: Các nội dung chính cần biết trong việc đưa đón học sinh là gì? \nContext: 10 \n - Biết quan tâm đ ến những người xung quanh.  \n- Luôn cởi mở tiếp nhận ý kiến đóng góp, liên t ục hoàn thi ện bản thân và có tinh th ần \nhọc hỏi để chất lượng dịch vụ ngày một tốt hơn.  \n3. Giọng đi ệu và l ời nói  \n- Giọng nói có âm lư ợng vừa phải, đủ nghe. Nói có ng ắt, nghỉ, có điểm dừng, điểm nhấn \nrõ ràng, tròn vành, r õ chữ, không nói ng ọng.  \n- Nói đầy đủ cả câu (câu có ch ủ ngữ, vị ngữ), có đủ các đại từ nhân xưng (không nói \ntrống không). Thư ờng xuyên s ử dụng những câu có ý nghĩa tích c ực trong giao ti ếp. \nTuyệt đối không nói t ục, chửi thề. Hạn chế dùng từ địa phương.  \nIII. CÁC QUY  ĐỊNH LAO Đ ỘNG \n1. Thời gian làm vi ệc \nKhung